# Basic preprocessing script for ATAC-seq data

In [1]:
import os
import sarge
from multiprocessing import Pool

In [2]:
os.chdir('../../../Rat-Pilot/atac-data/')

## Merge tag directories for visualization

In [3]:
'''
Define a function that can merge a tag directory.
'''
def merge_tag_dir(args):
    
    input_dirs,output_dir = args
    input_dir_string = ' '.join(input_dirs)
    
    cmd = f'makeTagDirectory {output_dir} -d {input_dir_string}'
    sarge.run(cmd)

In [4]:
tag_dir_cmds = [(('tag_directories/Core-r1','tag_directories/Core-r2'),'merged/Core'),
                (('tag_directories/IP-r1','tag_directories/IP-r2'),'merged/IP'),
                (('tag_directories/PFC-r1','tag_directories/PFC-r2'),'merged/PFC'),
                (('tag_directories/PL-r1','tag_directories/PL-r2'),'merged/PL'),
                (('tag_directories/Shell-r1','tag_directories/Shell-r2'),'merged/Shell'),
                (('tag_directories/Striatum-r1','tag_directories/Striatum-r2'),'merged/Striatum'),
                (('tag_directories/VTA-r1','tag_directories/VTA-r2'),'merged/VTA')]

In [5]:
%%time
pool = Pool(processes=len(tag_dir_cmds))
pool.map(merge_tag_dir,tag_dir_cmds)
pool.close()

CPU times: user 179 ms, sys: 57.4 ms, total: 237 ms
Wall time: 5min 13s


## Create a bigwig hub

In [6]:
%%time
%%bash
makeMultiWigHub.pl Rat-Pilot-ATAC rn6 -url http://homer.ucsd.edu/sjroth/Rat-Pilot-ATAC \
    -webdir /homer_data/www/html/sjroth/Rat-Pilot-ATAC -force -d merged/*

Colors that will be used (change with -color or -gradient options):
	Index	Color	Neg. color	Tag Directory
	1	255,150,150	(255,180,180)	merged/Core
	2	150,150,255	(180,180,255)	merged/IP
	3	150,255,150	(180,255,180)	merged/PFC
	4	255,200,150	(255,210,180)	merged/PL
	5	150,255,220	(180,255,240)	merged/Shell
	6	200,150,255	(210,180,255)	merged/Striatum
	7	200,200,150	(210,210,170)	merged/VTA


	Once finished, you will want to upload the following hub URL:
		http://homer.ucsd.edu/sjroth/Rat-Pilot-ATAC/Rat-Pilot-ATAC/hub.txt

	If loading to the Wash U Epigenome Browser, use:
		http://homer.ucsd.edu/sjroth/Rat-Pilot-ATAC/Rat-Pilot-ATAC/washU.hub.txt


	Visualization fragment length = 87
	Output file: merged/Core/Core.ucsc.bigWig
	No need to remove tags to get desired file size
	Generating bedGraph for chr1
	Generating bedGraph for chr10
	Generating bedGraph for chr11
	Generating bedGraph for chr12
	Generating bedGraph for chr13
	Generating bedGraph for chr14
	Generating bedGraph for chr15
	G

CPU times: user 47.7 ms, sys: 19.3 ms, total: 67 ms
Wall time: 29min 58s


## Call peaks

In [7]:
'''
Define a function that can call peaks by varying parameters.
'''
def call_peaks(tag_dir):
    
    cmd = f'findPeaks {tag_dir} -style factor -o auto'
    sarge.run(cmd)

In [8]:
tag_dirs = [os.path.join('tag_directories',tag_dir) for tag_dir in os.listdir('tag_directories/')]

In [9]:
%%time
pool = Pool(processes=len(tag_dirs))
pool.map(call_peaks,tag_dirs)
pool.close()

CPU times: user 233 ms, sys: 121 ms, total: 354 ms
Wall time: 5min 55s


## Merge all peaks (in total and for each experimental set)

In [10]:
%%bash
mergePeaks tag_directories/*/peaks.txt > tag_directories/all_peaks.txt

	Max distance to merge: direct overlap required (-d given)
	Merging peaks... 
	Comparing tag_directories/Core-r1/peaks.txt (10437 total) and tag_directories/Core-r1/peaks.txt (10437 total)
	Comparing tag_directories/Core-r1/peaks.txt (10437 total) and tag_directories/Core-r2/peaks.txt (9006 total)
	Comparing tag_directories/Core-r1/peaks.txt (10437 total) and tag_directories/IP-r1/peaks.txt (8930 total)
	Comparing tag_directories/Core-r1/peaks.txt (10437 total) and tag_directories/IP-r2/peaks.txt (5980 total)
	Comparing tag_directories/Core-r1/peaks.txt (10437 total) and tag_directories/PFC-r1/peaks.txt (5214 total)
	Comparing tag_directories/Core-r1/peaks.txt (10437 total) and tag_directories/PFC-r2/peaks.txt (14590 total)
	Comparing tag_directories/Core-r1/peaks.txt (10437 total) and tag_directories/PL-r1/peaks.txt (6015 total)
	Comparing tag_directories/Core-r1/peaks.txt (10437 total) and tag_directories/PL-r2/peaks.txt (9879 total)
	Comparing tag_directories/Core-r1/peaks.txt (1043

In [11]:
merge_cmds = [(('tag_directories/Core-r1/peaks.txt','tag_directories/Core-r2/peaks.txt'),'tag_directories/Core.peaks.txt'),
              (('tag_directories/IP-r1/peaks.txt','tag_directories/IP-r2/peaks.txt'),'tag_directories/IP.peaks.txt'),
              (('tag_directories/PFC-r1/peaks.txt','tag_directories/PFC-r2/peaks.txt'),'tag_directories/PFC.peaks.txt'),
              (('tag_directories/PL-r1/peaks.txt','tag_directories/PL-r2/peaks.txt'),'tag_directories/PL.peaks.txt'),
              (('tag_directories/Shell-r1/peaks.txt','tag_directories/Shell-r2/peaks.txt'),'tag_directories/Shell.peaks.txt'),
              (('tag_directories/Striatum-r1/peaks.txt','tag_directories/Striatum-r2/peaks.txt'),'tag_directories/Striatum.peaks.txt'),
              (('tag_directories/VTA-r1/peaks.txt','tag_directories/VTA-r2/peaks.txt'),'tag_directories/VTA.peaks.txt')]

In [12]:
for peak_files, output in merge_cmds:
    peak_file1,peak_file2 = peak_files
    !mergePeaks {peak_file1} {peak_file2} > {output}

	Max distance to merge: direct overlap required (-d given)
	Merging peaks... 
	Comparing tag_directories/Core-r1/peaks.txt (10437 total) and tag_directories/Core-r1/peaks.txt (10437 total)
	Comparing tag_directories/Core-r1/peaks.txt (10437 total) and tag_directories/Core-r2/peaks.txt (9006 total)
	Comparing tag_directories/Core-r2/peaks.txt (9006 total) and tag_directories/Core-r1/peaks.txt (10437 total)
	Comparing tag_directories/Core-r2/peaks.txt (9006 total) and tag_directories/Core-r2/peaks.txt (9006 total)

tag_directories/Core-r1/peaks.txt	tag_directories/Core-r2/peaks.txt	Total	Name
	X	3070	tag_directories/Core-r2/peaks.txt
X		4500	tag_directories/Core-r1/peaks.txt
X	X	5936	tag_directories/Core-r1/peaks.txt|tag_directories/Core-r2/peaks.txt
	Max distance to merge: direct overlap required (-d given)
	Merging peaks... 
	Comparing tag_directories/IP-r1/peaks.txt (8930 total) and tag_directories/IP-r1/peaks.txt (8930 total)
	Comparing tag_directories/IP-r1/peaks.txt (8930 total) an

## Get Distal Peaks for all peak files.

In [13]:
'''
Define a function that can isolate the distal peaks.
'''
def get_distal_peaks(peakfile):
    
    cmd = f'getDistalPeaks.pl {peakfile} rn6 > {peakfile[:-3]}distal.txt'
    sarge.run(cmd)

In [14]:
peak_files = []
for f in os.listdir('tag_directories/'):
    if f[-3:] == 'txt':
        peak_files.append(os.path.join('tag_directories',f))
peak_files

['tag_directories/PL.peaks.txt',
 'tag_directories/all_peaks.txt',
 'tag_directories/Core.peaks.txt',
 'tag_directories/Shell.peaks.txt',
 'tag_directories/Striatum.peaks.txt',
 'tag_directories/VTA.peaks.txt',
 'tag_directories/IP.peaks.txt',
 'tag_directories/PFC.peaks.txt']

In [15]:
pool = Pool(processes=len(peak_files))
pool.map(get_distal_peaks,peak_files)
pool.close()

## Convert all peak files to BED format

In [16]:
bed_cmds = []
for f in os.listdir('tag_directories/'):
    if f[-3:] == 'txt':
        bed_cmds.append([os.path.join('tag_directories',f),os.path.join('tag_directories',f[:-3]+'bed')])
bed_cmds

[['tag_directories/PL.peaks.txt', 'tag_directories/PL.peaks.bed'],
 ['tag_directories/Striatum.peaks.distal.txt',
  'tag_directories/Striatum.peaks.distal.bed'],
 ['tag_directories/Core.peaks.distal.txt',
  'tag_directories/Core.peaks.distal.bed'],
 ['tag_directories/VTA.peaks.distal.txt',
  'tag_directories/VTA.peaks.distal.bed'],
 ['tag_directories/all_peaks.txt', 'tag_directories/all_peaks.bed'],
 ['tag_directories/all_peaks.distal.txt',
  'tag_directories/all_peaks.distal.bed'],
 ['tag_directories/Core.peaks.txt', 'tag_directories/Core.peaks.bed'],
 ['tag_directories/PFC.peaks.distal.txt',
  'tag_directories/PFC.peaks.distal.bed'],
 ['tag_directories/Shell.peaks.distal.txt',
  'tag_directories/Shell.peaks.distal.bed'],
 ['tag_directories/PL.peaks.distal.txt',
  'tag_directories/PL.peaks.distal.bed'],
 ['tag_directories/Shell.peaks.txt', 'tag_directories/Shell.peaks.bed'],
 ['tag_directories/Striatum.peaks.txt', 'tag_directories/Striatum.peaks.bed'],
 ['tag_directories/IP.peaks.dist

In [17]:
for in_file,out_file in bed_cmds:
    !pos2bed.pl {in_file} > {out_file}


	Converted 11120 peaks total


	Converted 17025 peaks total


	Converted 7632 peaks total


	Converted 2900 peaks total


	Converted 37633 peaks total


	Converted 28571 peaks total


	Converted 13506 peaks total


	Converted 9748 peaks total


	Converted 5555 peaks total


	Converted 5771 peaks total


	Converted 11053 peaks total


	Converted 24662 peaks total


	Converted 4956 peaks total


	Converted 6814 peaks total


	Converted 10365 peaks total


	Converted 15752 peaks total



## Perform motif finding

In [18]:
%%time
%%bash
batchFindMotifsGenome.pl rn6 -size 500 -cpu 8 -p 10 \
-preparsedDir /gpfs/data01/bennerlab/home/sjroth/software/homer/data/genomes/rn6/preparsed -f tag_directories/*.txt

	14083 finished
	14092 finished
	14088 finished
	14090 finished
	14082 finished
	14079 finished
	14086 finished
	14080 finished
	57264 finished


findMotifsGenome.pl "tag_directories/all_peaks.distal.txt" rn6 "Motifs-tag_directories/all_peaks.distal.txt"  -size 500 -p 10 -preparsedDir /gpfs/data01/bennerlab/home/sjroth/software/homer/data/genomes/rn6/preparsed
findMotifsGenome.pl "tag_directories/all_peaks.txt" rn6 "Motifs-tag_directories/all_peaks.txt"  -size 500 -p 10 -preparsedDir /gpfs/data01/bennerlab/home/sjroth/software/homer/data/genomes/rn6/preparsed
findMotifsGenome.pl "tag_directories/Core.peaks.distal.txt" rn6 "Motifs-tag_directories/Core.peaks.distal.txt"  -size 500 -p 10 -preparsedDir /gpfs/data01/bennerlab/home/sjroth/software/homer/data/genomes/rn6/preparsed
findMotifsGenome.pl "tag_directories/Core.peaks.txt" rn6 "Motifs-tag_directories/Core.peaks.txt"  -size 500 -p 10 -preparsedDir /gpfs/data01/bennerlab/home/sjroth/software/homer/data/genomes/rn6/preparsed
findMotifsGenome.pl "tag_directories/IP.peaks.distal.txt" rn6 "Motifs-tag_directories/IP.peaks.distal.txt"  -size 500 -p 10 -preparsedDir /gpfs/data01/benne

CPU times: user 1.67 s, sys: 502 ms, total: 2.18 s
Wall time: 1h 34min 48s


## Get Expression Values for peaks

In [23]:
%%time 
%%bash
annotatePeaks.pl tag_directories/all_peaks.txt rn6 -raw -cpu 14 -d tag_directories/*r[1-2]* > tag_directories/all_peaks.raw.txt


	Peak file = tag_directories/all_peaks.txt
	Genome = rn6
	Organism = rat
	Will NOT normalize tag counts
	Will use up to 14 CPUs in parts that can use them
	Tag Directories:
		tag_directories/Core-r1
		tag_directories/Core-r2
		tag_directories/IP-r1
		tag_directories/IP-r2
		tag_directories/PFC-r1
		tag_directories/PFC-r2
		tag_directories/PL-r1
		tag_directories/PL-r2
		tag_directories/Shell-r1
		tag_directories/Shell-r2
		tag_directories/Striatum-r1
		tag_directories/Striatum-r2
		tag_directories/VTA-r1
		tag_directories/VTA-r2
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 37633
		Duplicated Peak IDs: 0

	Peak File Statistics:
		Total Peaks: 37633
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Reading Positions...
	-----------------------
	Fin

CPU times: user 3.72 ms, sys: 11.8 ms, total: 15.5 ms
Wall time: 1min 35s


In [24]:
%%time 
%%bash
annotatePeaks.pl tag_directories/all_peaks.txt rn6 -rlog -cpu 14 -d tag_directories/*r[1-2]* > tag_directories/all_peaks.rlog.txt


	Peak file = tag_directories/all_peaks.txt
	Genome = rn6
	Organism = rat
	Normalizing with rlog (R must be installed with package DESeq2)
	Will use up to 14 CPUs in parts that can use them
	Tag Directories:
		tag_directories/Core-r1
		tag_directories/Core-r2
		tag_directories/IP-r1
		tag_directories/IP-r2
		tag_directories/PFC-r1
		tag_directories/PFC-r2
		tag_directories/PL-r1
		tag_directories/PL-r2
		tag_directories/Shell-r1
		tag_directories/Shell-r2
		tag_directories/Striatum-r1
		tag_directories/Striatum-r2
		tag_directories/VTA-r1
		tag_directories/VTA-r2
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 37633
		Duplicated Peak IDs: 0

	Peak File Statistics:
		Total Peaks: 37633
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Reading Position

CPU times: user 14.1 ms, sys: 2.48 ms, total: 16.6 ms
Wall time: 1min 59s


In [25]:
%%time 
%%bash
annotatePeaks.pl tag_directories/all_peaks.distal.txt rn6 -raw -cpu 14 \
-d tag_directories/*r[1-2]* > tag_directories/all_peaks.distal.raw.txt


	Peak file = tag_directories/all_peaks.distal.txt
	Genome = rn6
	Organism = rat
	Will NOT normalize tag counts
	Will use up to 14 CPUs in parts that can use them
	Tag Directories:
		tag_directories/Core-r1
		tag_directories/Core-r2
		tag_directories/IP-r1
		tag_directories/IP-r2
		tag_directories/PFC-r1
		tag_directories/PFC-r2
		tag_directories/PL-r1
		tag_directories/PL-r2
		tag_directories/Shell-r1
		tag_directories/Shell-r2
		tag_directories/Striatum-r1
		tag_directories/Striatum-r2
		tag_directories/VTA-r1
		tag_directories/VTA-r2
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 28571
		Duplicated Peak IDs: 0

	Peak File Statistics:
		Total Peaks: 28571
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Reading Positions...
	---------------------

CPU times: user 5.21 ms, sys: 8.25 ms, total: 13.5 ms
Wall time: 1min 34s


In [26]:
%%time 
%%bash
annotatePeaks.pl tag_directories/all_peaks.distal.txt rn6 -rlog -cpu 14 \
-d tag_directories/*r[1-2]* > tag_directories/all_peaks.distal.rlog.txt


	Peak file = tag_directories/all_peaks.distal.txt
	Genome = rn6
	Organism = rat
	Normalizing with rlog (R must be installed with package DESeq2)
	Will use up to 14 CPUs in parts that can use them
	Tag Directories:
		tag_directories/Core-r1
		tag_directories/Core-r2
		tag_directories/IP-r1
		tag_directories/IP-r2
		tag_directories/PFC-r1
		tag_directories/PFC-r2
		tag_directories/PL-r1
		tag_directories/PL-r2
		tag_directories/Shell-r1
		tag_directories/Shell-r2
		tag_directories/Striatum-r1
		tag_directories/Striatum-r2
		tag_directories/VTA-r1
		tag_directories/VTA-r2
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 28571
		Duplicated Peak IDs: 0

	Peak File Statistics:
		Total Peaks: 28571
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Reading P

CPU times: user 9.84 ms, sys: 4.49 ms, total: 14.3 ms
Wall time: 1min 50s


In [27]:
peaks_to_tag_dir = [['tag_directories/Core.peaks.txt','tag_directories/Core-r1/','tag_directories/Core-r2/'],
                    ['tag_directories/Core.peaks.distal.txt','tag_directories/Core-r1/','tag_directories/Core-r2/'],
                    ['tag_directories/IP.peaks.txt','tag_directories/IP-r1/','tag_directories/IP-r2/'],
                    ['tag_directories/IP.peaks.distal.txt','tag_directories/IP-r1/','tag_directories/IP-r2/'],
                    ['tag_directories/PFC.peaks.txt','tag_directories/PFC-r1/','tag_directories/PFC-r2/'],
                    ['tag_directories/PFC.peaks.distal.txt','tag_directories/PFC-r1/','tag_directories/PFC-r2/'],
                    ['tag_directories/PL.peaks.txt','tag_directories/PL-r1/','tag_directories/PL-r2/'],
                    ['tag_directories/PL.peaks.distal.txt','tag_directories/PL-r1/','tag_directories/PL-r2/'],
                    ['tag_directories/Shell.peaks.txt','tag_directories/Shell-r1/','tag_directories/Shell-r2/'],
                    ['tag_directories/Shell.peaks.distal.txt','tag_directories/Shell-r1/','tag_directories/Shell-r2/'],
                    ['tag_directories/Striatum.peaks.txt','tag_directories/Striatum-r1/','tag_directories/Striatum-r2/'],
                    ['tag_directories/Striatum.peaks.distal.txt','tag_directories/Striatum-r1/','tag_directories/Striatum-r2/'],
                    ['tag_directories/VTA.peaks.txt','tag_directories/VTA-r1/','tag_directories/VTA-r2/'],
                    ['tag_directories/VTA.peaks.distal.txt','tag_directories/VTA-r1/','tag_directories/VTA-r2/']]

In [35]:
'''
Define a function that can get the expression of peaks.
'''
def get_exp(args):
    
    peakfile,tag_dir1,tag_dir2,norm = args
    
    cmd = f'annotatePeaks.pl {peakfile} rn6 -{norm} -cpu 2 -d {tag_dir1} {tag_dir2} > {peakfile[:-3]}{norm}.txt'
    sarge.run(cmd)

In [31]:
cmds = [x+['raw'] for x in peaks_to_tag_dir]+[x+['rlog'] for x in peaks_to_tag_dir]
cmds

[['tag_directories/Core.peaks.txt',
  'tag_directories/Core-r1/',
  'tag_directories/Core-r2/',
  'raw'],
 ['tag_directories/Core.peaks.distal.txt',
  'tag_directories/Core-r1/',
  'tag_directories/Core-r2/',
  'raw'],
 ['tag_directories/IP.peaks.txt',
  'tag_directories/IP-r1/',
  'tag_directories/IP-r2/',
  'raw'],
 ['tag_directories/IP.peaks.distal.txt',
  'tag_directories/IP-r1/',
  'tag_directories/IP-r2/',
  'raw'],
 ['tag_directories/PFC.peaks.txt',
  'tag_directories/PFC-r1/',
  'tag_directories/PFC-r2/',
  'raw'],
 ['tag_directories/PFC.peaks.distal.txt',
  'tag_directories/PFC-r1/',
  'tag_directories/PFC-r2/',
  'raw'],
 ['tag_directories/PL.peaks.txt',
  'tag_directories/PL-r1/',
  'tag_directories/PL-r2/',
  'raw'],
 ['tag_directories/PL.peaks.distal.txt',
  'tag_directories/PL-r1/',
  'tag_directories/PL-r2/',
  'raw'],
 ['tag_directories/Shell.peaks.txt',
  'tag_directories/Shell-r1/',
  'tag_directories/Shell-r2/',
  'raw'],
 ['tag_directories/Shell.peaks.distal.txt',
 

In [36]:
%%time
pool = Pool(processes=int(len(cmds)/2))
pool.map(get_exp,cmds)
pool.close()

CPU times: user 157 ms, sys: 80 ms, total: 237 ms
Wall time: 3min 4s


## Get coverage at promoters

In [2]:
os.chdir('../../../Rat-Pilot/')

In [3]:
!annotatePeaks.pl tss rn6 -size 2000 -hist 5 -cpu 50 \
-d atac-data/tag_directories/*r[1-2]* data/tag_directories/H3K27ac* > tss.coverage.txt


	Peak file = tss
	Genome = rn6
	Organism = rat
	Peak Region set to 2000
	-----------------------------------------------------
	Histogram mode activated (bin size = 5 bp)
	-----------------------------------------------------
	Will use up to 50 CPUs in parts that can use them
	Tag Directories:
		atac-data/tag_directories/Core-r1
		atac-data/tag_directories/Core-r2
		atac-data/tag_directories/IP-r1
		atac-data/tag_directories/IP-r2
		atac-data/tag_directories/PFC-r1
		atac-data/tag_directories/PFC-r2
		atac-data/tag_directories/PL-r1
		atac-data/tag_directories/PL-r2
		atac-data/tag_directories/Shell-r1
		atac-data/tag_directories/Shell-r2
		atac-data/tag_directories/Striatum-r1
		atac-data/tag_directories/Striatum-r2
		atac-data/tag_directories/VTA-r1
		atac-data/tag_directories/VTA-r2
		data/tag_directories/H3K27ac1-IL
		data/tag_directories/H3K27ac2-PL
		data/tag_directories/H3K27ac4-ST
		data/tag_directories/H3K27ac5-AC
		data/tag_directories/H3K27ac6-AS
		data/tag_directories/H3K2